In [29]:
import JSSP
import gym
import time
import numpy as np

# Environment Initialization

In [30]:
env_name = "JSSP-v0"
env = gym.make(env_name, instance_path = "instance1.txt")
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

Observation space: Dict(0:Box([0 0], [2 2], (2,), int64), 1:Box([-2 -2], [2 2], (2,), int64))
Action space: Box([-1 -1], [2 2], (2,), int64)


# Action Choosing

In [31]:
def get_legal_action(jsp_env):
    is_legal = False
    while not is_legal:
        Chosen_action = jsp_env.action_space.sample()
        is_legal = jsp_env.is_legal(Chosen_action)
    return Chosen_action

# Baselines

## 1. Random Sampling

In [43]:
state = env.reset()
episodes = 50
max_score = -100000
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = get_legal_action(env)
        # if not (np.all(action == -1)):
        #     print('Episode:{} Action:{} Time:{}'.format(episode, action, env.time))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Total_reward:{}'.format(episode, score))
    max_score = max(max_score, score)
print('From {} Episodes best policy has reward {}'.format(episodes, max_score))

Episode:1 Total_reward:-77
Episode:2 Total_reward:-64
Episode:3 Total_reward:-77
Episode:4 Total_reward:-69
Episode:5 Total_reward:-65
Episode:6 Total_reward:-73
Episode:7 Total_reward:-65
Episode:8 Total_reward:-74
Episode:9 Total_reward:-83
Episode:10 Total_reward:-64
Episode:11 Total_reward:-76
Episode:12 Total_reward:-76
Episode:13 Total_reward:-65
Episode:14 Total_reward:-76
Episode:15 Total_reward:-70
Episode:16 Total_reward:-64
Episode:17 Total_reward:-58
Episode:18 Total_reward:-65
Episode:19 Total_reward:-65
Episode:20 Total_reward:-60
Episode:21 Total_reward:-78
Episode:22 Total_reward:-62
Episode:23 Total_reward:-65
Episode:24 Total_reward:-43
Episode:25 Total_reward:-50
Episode:26 Total_reward:-58
Episode:27 Total_reward:-75
Episode:28 Total_reward:-68
Episode:29 Total_reward:-71
Episode:30 Total_reward:-75
Episode:31 Total_reward:-52
Episode:32 Total_reward:-76
Episode:33 Total_reward:-74
Episode:34 Total_reward:-59
Episode:35 Total_reward:-51
Episode:36 Total_reward:-50
E

## 2. Q-Learning

In [ ]:
# 1. Load Environment and Q-table structure
env_name = "JSSP-v0"
env = gym.make(env_name, instance_path = "instance1.txt")
Q = {}
# 2. Parameters of Q-learning
eta = .628
gma = .9
epis = 50
rev_list = [] # rewards per episode calculate
# 3. Q-learning Algorithm
for i in range(epis):
    # Reset environment
    s = env.reset()
    rAll = 0
    d = False
    # The Q-Table learning algorithm
    while not d:
        # env.render()
        # Choose action from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state & reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + eta*(r + gma*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
    rev_list.append(rAll)
    print("Episode: " + str(i) + " has reward " + str(rAll))
    # env.render()
# Code will stop at d == True, and render one state before it
print("Reward Sum on all episodes " + str(sum(rev_list)/epis))
print("Final Values Q-Table")
print(Q)